In [1]:
import pandas as pd

# load data
data = pd.read_csv("CLEAN_DATA/Data_europe_2005_2020_ps.csv")

# creating function to categorize political orientation
def categorize_orientation_party(rile):
    if rile < 0:
        return 'Left'
    elif rile > 0:
        return 'Right'
        
data['Political Orientation Party'] = data['rile'].apply(categorize_orientation_party)

# success rate for Left and Right parties
data['Success'] = data['absseat'].apply(lambda x: 1 if x > 0 else 0)
success_rate_by_orientation_country = data.groupby(['countryname', 'Political Orientation Party'])['Success'].mean().reset_index()

# Pivot for success rate
success_rate_pivot = success_rate_by_orientation_country.pivot(index='countryname', columns='Political Orientation Party', values='Success').reset_index()
success_rate_pivot.columns = ['Country', 'Success Rate - Left Parties', 'Success Rate - Right Parties']

# Calculate average stance positivity score per political orientation within each country
avg_stance_by_orientation = data.groupby(['countryname', 'Political Orientation Party'])['stance_positivity_score'].mean().reset_index()

# Pivot for stance positivity score
avg_stance_pivot = avg_stance_by_orientation.pivot(index='countryname', columns='Political Orientation Party', values='stance_positivity_score').reset_index()
avg_stance_pivot.columns = ['Country', 'Avg Positivity Score - Left Parties', 'Avg Positivity Score - Right Parties']

# Merge average stance positivity score and success rate tables
country_profiles = avg_stance_pivot.merge(success_rate_pivot, on='Country', how='left')
country_profiles.fillna(0, inplace=True)

# Determine Comparative Success Rate
def compare_success_rates(row):
    if row['Success Rate - Left Parties'] > row['Success Rate - Right Parties']:
        return '+'
    elif row['Success Rate - Left Parties'] < row['Success Rate - Right Parties']:
        return '-'
    else:
        return '±'

country_profiles['Comparative Success Rate - Left'] = country_profiles.apply(compare_success_rates, axis=1)
country_profiles['Comparative Success Rate - Right'] = country_profiles['Comparative Success Rate - Left'].apply(lambda x: '+' if x == '-' else ('-' if x == '+' else '±'))

# Prepare the final DataFrame
final_df = country_profiles[['Country', 'Avg Positivity Score - Left Parties', 'Comparative Success Rate - Left', 'Avg Positivity Score - Right Parties', 'Comparative Success Rate - Right']]
final_df.rename(columns={'Avg Positivity Score - Left Parties': 'Stances - Left', 'Comparative Success Rate - Left': 'Success Rate - Left', 'Avg Positivity Score - Right Parties': 'Stances - Right', 'Comparative Success Rate - Right': 'Success Rate - Right'}, inplace=True)

# convert positivity scores to '+' or '-'
def convert_positivity(score):
    return '+' if score > 0 else '-'

final_df['Stances - Left'] = final_df['Stances - Left'].apply(convert_positivity)
final_df['Stances - Right'] = final_df['Stances - Right'].apply(convert_positivity)

# Assign country profile labels
unique_profiles = final_df.drop('Country', axis=1).drop_duplicates()
profile_labels = {tuple(row): 'Country ' + chr(65+i) for i, row in enumerate(unique_profiles.itertuples(index=False, name=None))}
final_df['Country Profile'] = final_df.drop('Country', axis=1).apply(tuple, axis=1).map(profile_labels)


# Group countries by 'Country Profile' and aggregate them into lists
profile_to_countries = final_df.groupby('Country Profile')['Country'].apply(list).reset_index()

# Drop the 'country' column
final_df.drop(columns=['Country'], inplace=True)

# Sort the DataFrame by the 'Unique Country Profile' column
final_df.sort_values(by='Country Profile', inplace=True)

# Drop duplicates
final_df.drop_duplicates(subset=['Country Profile'], keep='first', inplace=True)

# Move Unique Country Profile to the front
final_df = final_df[ ['Country Profile'] + [ col for col in final_df.columns if col != 'Country Profile' ] ]
final_df = final_df.merge(profile_to_countries, on='Country Profile')

# Convert the lists in the 'Country' column to strings without brackets
final_df['Country'] = final_df['Country'].apply(lambda x: ', '.join(x))

# Set the maximum column width to None to prevent truncation
pd.set_option('display.max_colwidth', None)

# Display the modified DataFrame
display(final_df)

# turn dataframe into csv file
final_df.to_csv("CLEAN_DATA/country_profiles.csv")




Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/bb/hkfpdvbn46vcqvd2x75jvk0h0000gn/T/ipykernel_36112/139276148.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'Avg Positivity Score - Left Parties': 'Stances - Left', 'Comparative Success Rate - Left': 'Success Rate - Left', 'Avg Positivity Score - Right Parties': 'Stances - Right', 'Comparative Success Rate - Right': 'Success Rate - Right'}, inplace=True)
/var/folders/bb/hkfpdvbn46vcqvd2x75jvk0h0000gn/T/ipykernel_36112/139276148.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Stances - Left'] = final_df['

,Country Profile,Stances - Left,Success Rate - Left,Stances - Right,Success Rate - Right,Country
0,Country A,+,-,+,+,"Austria, Iceland, Poland, Slovakia, Spain, Switzerland"
1,Country B,+,±,+,±,"Belgium, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Germany, Hungary, Ireland, Italy, Latvia, Lithuania, Netherlands, Norway, Portugal, Romania, Serbia, Slovenia, Sweden, Ukraine"
2,Country C,-,-,+,+,Bulgaria
3,Country D,+,+,+,-,"Greece, Moldova, Montenegro, United Kingdom"
